# Create Project

In [1]:
# Install tmdbsimple (only need to run once)
!pip install tmdbsimple

In [2]:
 !pip install tqdm


## Load libraries

In [11]:
# import libraries
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import StrMethodFormatter
import matplotlib.ticker as ticker

#import 

# API tools and json
import json
import tmdbsimple as tmdb
from tqdm.notebook import trange, tqdm
from tqdm.notebook import tqdm_notebook 



# settings
pd.options.display.float_format = '{:,.2f}'.format

## Functions

In [6]:
def get_movie_with_rating(movie_id):
    #get the movie and release dates
    movie = tmdb.Movies(movie_id)
    ##contruct output dict
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [7]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename, 'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        file.seek(0)
        json.dump(file_data, file)

## Load data

In [9]:
# open the json
with open("C:\\Users\\Elizabeth Spreng\\.secret\\tmbd_api.json", 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [12]:
# load tmbd simple
tmdb.API_KEY =  login['api-key']

## Test functions
- This test will check that the functions are running correctly for 2 movie ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook").

In [13]:
# using function
avengers = get_movie_with_rating('tt0848228')
notebook = get_movie_with_rating('tt0332280')
